In [1]:
import sys, os
sys.path.insert(0, '../..')
# os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import numpy as np
import pandas as pd
import torch
from models.weighted_node2vec import WeightedNode2Vec
from dataset import triplet_dataset, pokec_data
from utils.config import DEVICE
import gc
from tqdm import trange, tqdm
from utils.link_prediction import GCNLinkPrediction
import residual2vec as rv
import warnings
import faiss
import matplotlib.pyplot as plt
from utils.score import statistical_parity
from utils import graph_utils
from utils.snakemake_utils import FileResources
import seaborn as sns
from utils import snakemake_utils
import pickle as pkl
warnings.filterwarnings("ignore")
gc.enable()

/data/sg/ashutiwa/miniconda3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using GPU: cuda:0


In [3]:
y = snakemake_utils.get_dataset("pokec").get_grouped_col()
sp_score, roc_score, gini_score, embs = {}, {}, {}, {}
DATA_ROOT = '/data/sg/ashutiwa/final_results/pokec/'
! ls /data/sg/ashutiwa/final_results/airport/
DIRS = ['data_one/', 'data_two/', 'data_three/', 'data_four/', 'data_five/']
DATASET = 'pokec'
M = ["GCN_baseline", "GCN_r2v", 
     "GAT_baseline","GAT_r2v"]

TEST_ADJ_FILE='/data/sg/ashutiwa/final_results/pokec/data_one/pokec_test_adj.npz'
NUM_NODES = 1632803
k=5

Using existing file soc-pokec-profiles.txt.gz
Using existing file soc-pokec-relationships.txt.gz


data_five  data_one    data_two        roc_score.pkl
data_four  data_three  gini_score.pkl  sp_score.pkl


# distributions of positive and negative edgeswith dot product

In [4]:
def get_roc_score(df):
    from sklearn.metrics import roc_auc_score
    y_pred = np.concatenate((df.positive_dot.values, df.negative_dot.values))
    y_true = np.concatenate((np.ones_like(df.positive_dot.values), np.zeros_like(df.negative_dot.values)))
    return roc_auc_score(y_true, y_pred)



In [5]:
def plot_dot_(df, title):
#     df = pd.DataFrame({
#         "positive_dot": np.array([np.dot(a[i], p[i]) for i in trange(a.shape[0])]),
#         "negative_dot": np.array([np.dot(a[i], n[i]) for i in trange(a.shape[0])])
#     })
    
#     fig, ax = plt.subplots(1,1)
    df.positive_dot.plot(kind='density', title=title)
    df.negative_dot.plot(kind='density', title=title)
#     ax.set_xlim(-1000, 1000)
    ax.legend(['positive_neighbors', 'negative_neighbors'])
#     fig.show()

In [6]:
def create_dot_df(A, file=TEST_ADJ_FILE):
    """
    A are node embeddings of shape # nodes \times 128
    """
    EDGE_IDX = snakemake_utils.get_edge_index_from_sparse_path(file)
    NEGATIVE_EDGE_IDX, EDGE_IDX = graph_utils._negative_sampling_sparse(edge_index=EDGE_IDX, n_nodes=NUM_NODES, return_pos_samples=True)
    print("selected edge_idx size: {}".format(EDGE_IDX.size(1)))

    positive_dot = np.sum(A[EDGE_IDX[0,:]] * A[EDGE_IDX[1,:]], axis=1)
    negative_dot = np.sum(A[NEGATIVE_EDGE_IDX[0,:]] * A[NEGATIVE_EDGE_IDX[1,:]], axis=1)
    dot = pd.DataFrame()
    dot["positive_dot"] = positive_dot
    dot["negative_dot"] = negative_dot
    
    return dot

# GCN ( baseline)

In [7]:
# model = "GCN_baseline"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + '_gcn_node2vec_embs.npy')
# A = embs[model]
# A.shape

In [8]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [9]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GCN ( r2v)

In [10]:
# model = "GCN_r2v"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET  + '_gcn_r2v_node2vec_embs.npy')
# A = embs[model]
# A.shape

In [11]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [12]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GAT ( baseline)

In [13]:
# model = "GAT_baseline"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + '_gat_node2vec_embs.npy')
# A = embs[model]

# A.shape

In [14]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [15]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GAT ( r2v)

In [16]:
# model = "GAT_r2v"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + '_gat_r2v_node2vec_embs.npy')
# A = embs[model]
# A.shape

In [17]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [18]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# plot statistical parity scores

# gini sp scores

In [19]:

# for i, j in gini_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(gini_score.keys()), y=list(gini_score.values()))
# ax.set_ylim(min(gini_score.values()), max(gini_score.values()))
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("gini statistical parity scores")

# sp scores

In [20]:

# for i, j in sp_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(sp_score.keys()), y=list(sp_score.values()))
# ax.set_ylim(min(sp_score.values()), max(sp_score.values()))
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("statistical parity scores")

# roc auc scores

In [21]:

# for i, j in roc_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(roc_score.keys()), y=list(roc_score.values()))
# ax.set_ylim(min(roc_score.values()), max(roc_score.values()))
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("roc scores")

In [22]:

# N = 20

# sp_scores, gini_scores = np.zeros((N, len(M))), np.zeros((N, len(M)))
# for k in range(N):
#     for idx, m in enumerate(M):
#         if k == 0: continue
#         batch_size =  2000
#         A = embs[m]
#         edges = snakemake_utils.return_new_graph(A, k, )
        
    
#         gini_scores[k, idx] = statistical_parity(edges, y, metric='gini')
#         sp_scores[k, idx] = statistical_parity(edges, y, )


In [23]:
# for i in range(len(M)):
#     plt.plot(gini_scores[:, i], label=M[i])
# plt.legend(loc="upper right")
# plt.title("gini scores vs # nn")

In [24]:
# for i in range(len(M)):
#     plt.plot(sp_scores[:, i], label=M[i])
# plt.legend(loc="upper right")

# plt.title("sp scores vs # nn")

# combine all different runs

In [25]:
MODEL_NAME_2_FILE = {
    "GCN_baseline": "_gcn_node2vec_embs.npy",
#     "GCN_baseline_crosswalk": "_crosswalk_gcn_node2vec_embs.npy",
    "GCN_r2v": "_gcn_r2v_node2vec_embs.npy",
#     "GCN_r2v_crosswalk": '_crosswalk_gcn_r2v_node2vec_embs.npy',
    "GAT_baseline": '_gat_node2vec_embs.npy',
#     "GAT_baseline_crosswalk": "_crosswalk_gat_node2vec_embs.npy",
    "GAT_r2v": "_gat_r2v_node2vec_embs.npy",
#     "GAT_r2v_crosswalk": "_crosswalk_gat_r2v_node2vec_embs.npy"
}
def roc_scores(model_name):
    scores = np.zeros(len(DIRS))
    for idx, d in enumerate(DIRS):
        A = np.load(DATA_ROOT + d + DATASET + MODEL_NAME_2_FILE[model_name])
        file = DATA_ROOT + d + DATASET + '_test_adj.npz'
        dot = create_dot_df(A, file)
        scores[idx] = get_roc_score(dot)
    return scores


def statisticalp_scores(model_name, K=5):
    scores, scores_gini = np.zeros(len(DIRS)), np.zeros(len(DIRS))
    for idx, d in enumerate(DIRS):
        A = np.load(DATA_ROOT + d + DATASET + MODEL_NAME_2_FILE[model_name])
        edges = snakemake_utils.return_new_graph(A, K)
        scores_gini[idx] = statistical_parity(edges, y, metric='gini')
        
        scores[idx] = statistical_parity(edges, y, )
    return scores, scores_gini

In [26]:

# roc_score = {m: np.mean(roc_scores(m)) for m in MODEL_NAME_2_FILE}

# for i, j in roc_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(roc_score.keys()), y=list(roc_score.values()))
# ax.set_ylim(min(roc_score.values()), max(roc_score.values()))
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("roc scores")

In [27]:

# temp = {m: statisticalp_scores(m) for m in MODEL_NAME_2_FILE}

# sp_score = {m: np.mean(v[0]) for m,v in temp.items()}
# for i, j in sp_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(sp_score.keys()), y=list(sp_score.values()))
# ax.set_ylim(min(sp_score.values()), max(sp_score.values()))
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("statistical parity scores")

In [28]:
# gini_score = {m: np.mean(v[1]) for m,v in temp.items()}
# for i, j in gini_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(gini_score.keys()), y=list(gini_score.values()))
# ax.set_ylim(min(gini_score.values()), max(gini_score.values()))
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("gini statistical parity scores")

In [29]:

# roc_score, sp_score, gini_score= {}, {}, {}
roc_score = {m: roc_scores(m) for m in tqdm(MODEL_NAME_2_FILE)}
temp = {m: statisticalp_scores(m) for m in tqdm(MODEL_NAME_2_FILE)}
sp_score = {m: v[0] for m,v in temp.items()}
gini_score = {m: v[1] for m,v in temp.items()}
pkl.dump(roc_score, open(DATA_ROOT + "roc_score.pkl", 'wb'))

pkl.dump(sp_score, open(DATA_ROOT + "sp_score.pkl", 'wb'))

pkl.dump(gini_score, open(DATA_ROOT + "gini_score.pkl", 'wb'))

  0%|                                                                                                                                   | 0/4 [00:00<?, ?it/s]

selected edge_idx size: 16286791
selected edge_idx size: 16285621
selected edge_idx size: 16284122
selected edge_idx size: 16286571
selected edge_idx size: 16285607


 25%|█████████████████████████████▊                                                                                         | 1/4 [43:48<2:11:25, 2628.63s/it]

selected edge_idx size: 16286791
selected edge_idx size: 16285621
selected edge_idx size: 16284122
selected edge_idx size: 16286571
selected edge_idx size: 16285607


 50%|██████████████████████████████████████████████████████████▌                                                          | 2/4 [1:24:48<1:24:18, 2529.09s/it]

...

 83%|█████████████████████████████████████████████████████████████████████████████████████████████████▊                    | 676/816 [00:06<00:01, 103.89it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 816/816 [00:07<00:00, 103.57it/s]

 51%|████████████████████████████████████████████████████████████▍                                                         | 418/816 [00:03<00:03, 107.26it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 816/816 [00:08<00:00, 99.96it/s]

 13%|███████████████▏                                                                                                      | 105/816 [00:00<00:06, 102.73it/s]


 79%|█████████████████████████████████████████████████████████████████████████████████████████████▋                         | 642/816 [00:06<00:01, 99.57it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [09:05<00:00, 136.45s/it]
